$$
    \newcommand{\genericdel}[3]{%
      \left#1#3\right#2
    }
    \newcommand{\del}[1]{\genericdel(){#1}}
    \newcommand{\sbr}[1]{\genericdel[]{#1}}
    \newcommand{\cbr}[1]{\genericdel\{\}{#1}}
    \newcommand{\abs}[1]{\genericdel||{#1}}
    \DeclareMathOperator*{\argmin}{arg\,min}
    \DeclareMathOperator*{\argmax}{arg\,max}
    \DeclareMathOperator{\Pr}{\mathbb{p}}
    \DeclareMathOperator{\E}{\mathbb{E}}
    \DeclareMathOperator{\Ind}{\mathbb{I}}
    \DeclareMathOperator{\var}{var}
    \DeclareMathOperator{\cov}{cov}
    \DeclareMathOperator{\invchi}{\mathrm{Inv-\chi}^2}
    \newcommand{\effect}{\mathrm{eff}}
    \newcommand{\xtilde}{\widetilde{X}}
    \DeclareMathOperator{\normal}{\mathcal{N}}
    \DeclareMathOperator{\unif}{Uniform}
    \DeclareMathOperator{\GP}{\mathcal{GP}}
    \newcommand{\Tn}{\mathrm{T}_{n}}
    \newcommand{\Tx}{\mathrm{T}_{x}}
    \newcommand{\station}[1]{\mathrm{station}\sbr{#1}}
    \newcommand{\xvec}{\mathbf{x}}
    \newcommand{\indep}{\perp}
    \newcommand{\iid}{iid}
    \newcommand{\trans}{^{\intercal}}
    \newcommand{\sigmaf}{\sigma_{\mathrm{GP}}}
    \newcommand{\sigman}{\sigma_{\epsilon}}
$$

In this notebook, we implement the much simpler model:
\begin{align}
    k_{st}(\xvec,\xvec',t,t') &= k_{time}(t,t') \cdot k_{space}(\xvec, \xvec') + k_{mean}(\xvec, \xvec'),,\\
    k_{space}(\xvec, \xvec') &= \sigmaf^2 \exp\del{-\frac{\del{\xvec-\xvec'}\trans\del{\xvec-\xvec'}}{2\ell_x^2}}\,,\\
    k_{time}(t, t') &= \exp\del{-\frac{\del{t-t'}^2}{2\ell_t^2}}\,,\\
    k_{mean}(\xvec, \xvec') &= \sigma_\mu^2 \exp\del{-\frac{\del{\xvec-\xvec'}\trans\del{\xvec-\xvec'}}{2\ell_\mu^2}}\,,\\
\end{align}

\begin{align}
    T_i &= f(\xvec_i, t_i) + \epsilon_i\\
    f(\xvec_i, t_i) &\sim \GP\del{0, k_{st}(\xvec,\xvec',t,t')}\\
    \epsilon_i &\overset{\iid}{\sim} \normal\del{0,\sigman^2}\\
\end{align}

We then add a diurnal component.

In [67]:
using TimeSeries
using DataFrames
import Proj4
using Optim
using Distances
using Statistics
using Printf
;

In [3]:
import PyPlot; plt=PyPlot
using LaTeXStrings
plt.rc("figure", dpi=300.0)
plt.rc("figure", figsize=(6,4))
plt.rc("savefig", dpi=300.0)
plt.rc("text", usetex=true)
plt.rc("font", family="serif")
plt.rc("font", serif="Palatino")
;

┌ Info: Recompiling stale cache file /Users/imolk/Library/Julia/alternative_depots/climate/compiled/v1.1/PyPlot/oatAj.ji for PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1184


In [2]:
using Revise
import TempModel
using GaussianProcesses

┌ Info: Recompiling stale cache file /Users/imolk/Library/Julia/alternative_depots/climate/compiled/v1.1/Revise/M1Qoh.ji for Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
└ @ Base loading.jl:1184


# Data Import and Preprocessing

In [9]:
include("iowa.jl")
data_dir = "../data"
iowa = prepare_iowa_data(data_dir);

┌ Warning: dropmissing! will change eltype of cols to disallow missing by default. Use dropmissing!(df, cols, disallowmissing=false) to retain missing.
│   caller = dropmissing!(::DataFrame) at abstractdataframe.jl:733
└ @ DataFrames /Users/imolk/Library/Julia/alternative_depots/climate/packages/DataFrames/IKMvt/src/abstractdataframe/abstractdataframe.jl:733


## Distances

To get distances between stations, we can either use a function to compute distances on a sphere, or we can first project the coordinates onto a Euclidean plane, and then compute normal distances. I'll do it both ways to check they're consistent (equal up to a multiplication constant), and then use Euclidean distances for convenience.

In [13]:
iowa[:isdSubset][1,:][:LAT]

41.883

In [14]:
numstations = nrow(iowa[:isdSubset])
pairwiseSphere = zeros(numstations, numstations)
for i in 1:numstations
    for j in 1:i
        if i==j
            continue
        end
        station1 = iowa[:isdSubset][i,:]
        station2 = iowa[:isdSubset][j,:]
        lat1=  station1[:LAT]
        lon1 = station1[:LON]
        lat2 = station2[:LAT]
        lon2 = station2[:LON]
        pairwiseSphere[i,j] = TempModel.distance_on_unit_sphere(lat1, lon1, lat2, lon2)
        pairwiseSphere[j,i] = pairwiseSphere[i,j]
    end
end
pairwiseSphere

4×4 Array{Float64,2}:
 0.0        0.0259496  0.0146736  0.0303475
 0.0259496  0.0        0.024088   0.0285853
 0.0146736  0.024088   0.0        0.0158124
 0.0303475  0.0285853  0.0158124  0.0      

In [15]:
pairwiseEuclid=pairwise(Euclidean(), Matrix(iowa[:isdSubset][[:X_PRJ,:Y_PRJ]])')

4×4 Array{Float64,2}:
      0.0        165736.0        93510.4        1.93474e5
 165736.0             0.0            1.53559e5  1.81942e5
  93510.4             1.53559e5      0.0        1.00846e5
      1.93474e5       1.81942e5      1.00846e5  0.0      

Ratio of the two distance matrices: close enough to a constant!

In [16]:
pairwiseEuclid ./ pairwiseSphere

4×4 Array{Float64,2}:
 NaN            6.38684e6    6.37271e6    6.37527e6
   6.38684e6  NaN            6.37493e6    6.36489e6
   6.37271e6    6.37493e6  NaN            6.37765e6
   6.37527e6    6.36489e6    6.37765e6  NaN        

# Product of SE kernels

In [143]:
function make_kernel()
    k_time = SEIso(0.0,0.0)
    k_spatial = SEIso(log(10^5), log(1.0))
    k_means = SEIso(log(10^4), log(10.0))

    k_spatiotemporal = Masked(k_time, [1]) * 
                       Masked(fix(k_spatial, :lσ), [2,3]) + 
                       fix(Masked(k_means, [2,3]))
    return Dict(
        :time => k_time,
        :spatial => k_spatial,
        :mean => k_means,
        :spatiotemporal => k_spatiotemporal
        )
end

function showkernel(k_time, k_spatial, logNoise)
    print("\nk: Temporal kernel \n=================\n")
    @printf("σ: %5.3f\n", √k_time.σ2)
    @printf("l: %5.3f hours\n", √k_time.ℓ2)
    print("\nk: Spatial kernel \n=================\n")
    @printf("σ: %5.3f\n", √k_spatial.σ2)
    @printf("l: %5.3f km\n", √k_spatial.ℓ2 / 1000)
    print("\n=================\n")
    @printf("σy: %5.3f\n", exp(logNoise))
end
;

In [80]:
kdict = make_kernel()
@time opt_out = TempModel.optim_kernel(kdict[:spatiotemporal], 0.0, iowa[:isdSubset], iowa[:hourly_data], :Optim);

In [82]:
print(opt_out[:hyp])

[-0.822288, 0.996802, 1.31718, 12.0805]

In [83]:
opt_out[:mll]

-55614.52002047768

In [84]:
print("\nk: Temporal kernel \n=================\n")
@printf("σ: %5.3f\n", √k_time.σ2)
@printf("l: %5.3f hours\n", √k_time.ℓ2)
print("\nk: Spatial kernel \n=================\n")
@printf("σ: %5.3f\n", √k_spatial.kernel.σ2)
@printf("l: %5.3f km\n", √k_spatial.kernel.ℓ2 / 1000)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(opt_out[:hyp][1]))


k: Temporal kernel 
σ: 3.733
l: 2.710 hours

k: Spatial kernel 
σ: 1.000
l: 176.391 km

σy: 0.439


## NLopt

Using an alternative optimizer.
The `:NLopt` method uses the L-BFGS implemented in [NLopt](https://nlopt.readthedocs.io/en/latest/), while the `:Optim` method uses the Conjugate Gradient Descent method of the julia [Optim package](https://github.com/JuliaNLSolvers/Optim.jl).

In [76]:
kdict = make_kernel()
@time nlopt_out = TempModel.optim_kernel(kdict[:spatiotemporal], 0.0, iowa[:isdSubset], iowa[:hourly_data], :NLopt)
;

creating GP chunks
begin optimization
213.687666 seconds (1.58 M allocations: 2.422 GiB, 0.55% gc time)


In [77]:
print(nlopt_out[:hyp])

[-0.82229, 0.996798, 1.31717, 12.0804]

In [78]:
nlopt_out[:mll]

-55614.52001889927

In [79]:
print("\nk: Temporal kernel \n=================\n")
@printf("σ: %5.3f\n", √k_time.σ2)
@printf("l: %5.3f hours\n", √k_time.ℓ2)
print("\nk: Spatial kernel \n=================\n")
@printf("σ: %5.3f\n", √k_spatial.kern.σ2)
@printf("l: %5.3f km\n", √k_spatial.kern.ℓ2 / 1000)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(nlopt_out[:hyp][1]))


k: Temporal kernel 
σ: 3.733
l: 2.710 hours

k: Spatial kernel 


ErrorException: type FixedKernel has no field kern

# Optim CV module

In [114]:
module OptimCV
    using GaussianProcesses
    using TempModel
    using TempModel: GPRealisations
    using GaussianProcesses: Folds, logp_CVfold, dlogpdθ_CVfold
    using DataFrames
    using Optim
    using Statistics
    
    function optim_kernel_CV(k_spatiotemporal::Kernel, logNoise_init::Float64, 
                          stations_data::DataFrame, hourly_data::DataFrame, 
                          method::Symbol=:NLopt; 
                          x_tol=1e-5, f_tol=1e-10)
        chunks=GPE[]
        chunk_width=24*10 # 10 days at a time
        tstart=0.0
        nobsv=0
        max_time = maximum(hourly_data[:ts_hours])
        println("creating GP chunks")
        folds_reals = Folds[]
        while tstart < max_time
            tend=tstart+chunk_width
            in_chunk= tstart .<= hourly_data[:ts_hours] .< tend
            hourly_chunk = hourly_data[in_chunk,:]
            nobsv_chunk = sum(in_chunk)
            nobsv += nobsv_chunk

            chunk_X_PRJ = stations_data[:X_PRJ][hourly_chunk[:station]]
            chunk_Y_PRJ = stations_data[:Y_PRJ][hourly_chunk[:station]]
            chunk_X = [hourly_chunk[:ts_hours] chunk_X_PRJ chunk_Y_PRJ]

            y = hourly_chunk[:temp]
            chunk = GPE(chunk_X', y, MeanConst(mean(y)), k_spatiotemporal, logNoise_init)
            push!(chunks, chunk)
            
            station = hourly_chunk[:station]
            chunk_folds = [findall(isequal(statuniq), station) 
                    for statuniq in unique(station)]
            push!(folds_reals, chunk_folds)

            tstart=tend
        end
        reals = TempModel.GPRealisations(chunks)
        local min_neg_ll
        local min_hyp
        local opt_out
        println("begin optimization")
        if method == :NLopt
            min_neg_ll, min_hyp, ret, count = TempModel.optimize_NLopt_CV(
                    reals, folds_reals,
                    domean=false, kern=true, noise=true,
                    x_tol=x_tol, f_tol=f_tol)
            opt_out = (min_neg_ll, min_hyp, ret, count)
            @assert ret ∈ (:SUCCESS, :FTOL_REACHED, :XTOL_REACHED)
        elseif method == :Optim
            opt_out = TempModel.optimize_CV!(reals, folds_reals;
                    domean=false, kern=true, noise=true,
                    options=Optim.Options(x_tol=x_tol, f_tol=f_tol)
                                         )
            min_hyp = Optim.minimizer(opt_out)
            min_neg_ll = Optim.minimum(opt_out)
            @assert Optim.converged(opt_out)
        else
            throw(MethodError())
        end
        return Dict(
            :hyp => min_hyp,
            :logNoise => reals.logNoise,
            :mll => -min_neg_ll,
            :opt_out => opt_out,
           )
    end
end
;

In [119]:
kdict = make_kernel()
@time opt_out_CV = OptimCV.optim_kernel_CV(kdict[:spatiotemporal], 0.0, iowa[:isdSubset], iowa[:hourly_data], 
                                          :Optim);
opt_out_CV

creating GP chunks
begin optimization
LinearAlgebra.PosDefException(-1)
LinearAlgebra.PosDefException(-1)
1526.391586 seconds (5.48 G allocations: 407.864 GiB, 19.05% gc time)


Dict{Symbol,Any} with 4 entries:
  :mll      => -52829.4
  :hyp      => [-0.837896, 0.899595, 1.26595, 11.9267]
  :logNoise => -0.837896
  :opt_out  => Results of Optimization Algorithm…

In [120]:
opt_out_CV[:opt_out]

Results of Optimization Algorithm
 * Algorithm: Conjugate Gradient
 * Starting Point: [0.0,0.0,0.0,11.512925464970229]
 * Minimizer: [-0.8378959875390904,0.8995954054399115, ...]
 * Minimum: 5.282936e+04
 * Iterations: 17
 * Convergence: true
   * |x - x'| ≤ 1.0e-05: false 
     |x - x'| = 1.68e-05 
   * |f(x) - f(x')| ≤ 1.0e-10 |f(x)|: true
     |f(x) - f(x')| = 8.99e-11 |f(x)|
   * |g(x)| ≤ 1.0e-08: false 
     |g(x)| = 5.95e-02 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 47
 * Gradient Calls: 29

In [125]:
opt_out_CV[:hyp]

4-element Array{Float64,1}:
 -0.8378959875390904
  0.8995954054399115
  1.265951854696949 
 11.926696343155182 

In [127]:
nlopt_out_CV[:hyp]

4-element Array{Float64,1}:
 -0.8378983734713557
  0.8995928402076627
  1.265952973549021 
 11.926695582873927 

In [122]:
kdict = make_kernel()
@time nlopt_out_CV = OptimCV.optim_kernel_CV(kdict[:spatiotemporal], 0.0, iowa[:isdSubset], iowa[:hourly_data], 
                                          :NLopt);
nlopt_out_CV

creating GP chunks
begin optimization
2362.857896 seconds (7.75 G allocations: 557.531 GiB, 17.71% gc time)


Dict{Symbol,Any} with 4 entries:
  :mll      => -52829.4
  :hyp      => [-0.837898, 0.899593, 1.26595, 11.9267]
  :logNoise => -0.837898
  :opt_out  => (52829.4, [-0.837898, 0.899593, 1.26595, 11.9267], :XTOL_REACHED…

In [145]:
kdict=make_kernel()
hyp = nlopt_out_CV[:hyp]
set_params!(kdict[:spatiotemporal], hyp[2:end])
showkernel(kdict[:time], kdict[:spatial], hyp[1])
;


k: Temporal kernel 
σ: 3.546
l: 2.459 hours

k: Spatial kernel 
σ: 1.000
l: 151.251 km

σy: 0.433


In [146]:
kdict=make_kernel()
hyp = nlopt_out[:hyp]
set_params!(kdict[:spatiotemporal], hyp[2:end])
showkernel(kdict[:time], kdict[:spatial], hyp[1])
;


k: Temporal kernel 
σ: 3.733
l: 2.710 hours

k: Spatial kernel 
σ: 1.000
l: 176.388 km

σy: 0.439
